Name: Pooja Meshram
Project : 2

#Load Data set

In [3]:
import pandas as pd
df = pd.read_csv ("amazon_alexa (1).tsv", sep="\t")

from google.colab import data_table
data_table.DataTable(df, num_rows_per_page=11)

,rating,date,variation,verified_reviews,feedback
0,5,31-Jul-18,Charcoal Fabric,Love my Echo!,1
1,5,31-Jul-18,Charcoal Fabric,Loved it!,1
2,4,31-Jul-18,Walnut Finish,"Sometimes while playing a game, you can answer...",1
3,5,31-Jul-18,Charcoal Fabric,I have had a lot of fun with this thing. My 4 ...,1
4,5,31-Jul-18,Charcoal Fabric,Music,1
...,...,...,...,...,...
3145,5,30-Jul-18,Black Dot,"Perfect for kids, adults and everyone in betwe...",1
3146,5,30-Jul-18,Black Dot,"Listening to music, searching locations, check...",1
3147,5,30-Jul-18,Black Dot,"I do love these things, i have them running my...",1
3148,5,30-Jul-18,White Dot,Only complaint I have is that the sound qualit...,1


#Import the required libraries

In [5]:
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
from sklearn.base import TransformerMixin
from sklearn.pipeline import Pipeline
from nltk.tokenize import word_tokenize
from gensim.models import Word2Vec
from gensim.models import KeyedVectors

In [9]:
df_amazon = pd.read_csv ("amazon_alexa (1).tsv", sep="\t")

In [10]:
df_amazon.head()

,rating,date,variation,verified_reviews,feedback
0,5,31-Jul-18,Charcoal Fabric,Love my Echo!,1
1,5,31-Jul-18,Charcoal Fabric,Loved it!,1
2,4,31-Jul-18,Walnut Finish,"Sometimes while playing a game, you can answer...",1
3,5,31-Jul-18,Charcoal Fabric,I have had a lot of fun with this thing. My 4 ...,1
4,5,31-Jul-18,Charcoal Fabric,Music,1


In [12]:
df_amazon_mod = df_amazon[['verified_reviews','feedback']]
df_amazon_mod.head()

,verified_reviews,feedback
0,Love my Echo!,1
1,Loved it!,1
2,"Sometimes while playing a game, you can answer...",1
3,I have had a lot of fun with this thing. My 4 ...,1
4,Music,1


##Remove punctuation & numerics



In [21]:
import re
def clean(string):
    string = str(string)
    cleanString = re.sub('[^A-Za-z]+',' ', string )
    return cleanString


In [22]:
df_amazon_mod

,verified_reviews,feedback
0,Love my Echo!,1
1,Loved it!,1
2,"Sometimes while playing a game, you can answer...",1
3,I have had a lot of fun with this thing. My 4 ...,1
4,Music,1
...,...,...
3145,"Perfect for kids, adults and everyone in betwe...",1
3146,"Listening to music, searching locations, check...",1
3147,"I do love these things, i have them running my...",1
3148,Only complaint I have is that the sound qualit...,1


from matplotlib import pyplot as plt
df_amazon_mod['feedback'].plot(kind='hist', bins=20, title='feedback')
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
df_amazon_mod['feedback'].plot(kind='line', figsize=(8, 4), title='feedback')
plt.gca().spines[['top', 'right']].set_visible(False)

##Trim out words with two or less characters


In [23]:
df['reviews_len_trim'] = df.reviews_non_numeric.map(lambda x: ' '.join([w for w in x.split() if len(w)>2]))
df.head()

,verified_reviews,feedback,reviews_non_numeric,reviews_len_trim
0,Love my Echo!,1,Love my Echo,Love Echo
1,Loved it!,1,Loved it,Loved
2,"Sometimes while playing a game, you can answer...",1,Sometimes while playing a game you can answer ...,Sometimes while playing game you can answer qu...
3,I have had a lot of fun with this thing. My 4 ...,1,I have had a lot of fun with this thing My yr ...,have had lot fun with this thing old learns ab...
4,Music,1,Music,Music


##Force lowercase and tokenize

In [24]:
df['reviews_tokenized'] = df.reviews_len_trim.map(lambda x: x.lower().split())
df.head()

,verified_reviews,feedback,reviews_non_numeric,reviews_len_trim,reviews_tokenized
0,Love my Echo!,1,Love my Echo,Love Echo,"[love, echo]"
1,Loved it!,1,Loved it,Loved,[loved]
2,"Sometimes while playing a game, you can answer...",1,Sometimes while playing a game you can answer ...,Sometimes while playing game you can answer qu...,"[sometimes, while, playing, game, you, can, an..."
3,I have had a lot of fun with this thing. My 4 ...,1,I have had a lot of fun with this thing My yr ...,have had lot fun with this thing old learns ab...,"[have, had, lot, fun, with, this, thing, old, ..."
4,Music,1,Music,Music,[music]


## Remove stopwords

In [25]:
import nltk
nltk.download('stopwords')

from nltk.corpus import stopwords
stop_words=set(stopwords.words('english'))

df['reviews_tokenized_stop'] = df.reviews_tokenized.map(lambda x: [w for w in x if not w in stop_words])
df.head()

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


,verified_reviews,feedback,reviews_non_numeric,reviews_len_trim,reviews_tokenized,reviews_tokenized_stop
0,Love my Echo!,1,Love my Echo,Love Echo,"[love, echo]","[love, echo]"
1,Loved it!,1,Loved it,Loved,[loved],[loved]
2,"Sometimes while playing a game, you can answer...",1,Sometimes while playing a game you can answer ...,Sometimes while playing game you can answer qu...,"[sometimes, while, playing, game, you, can, an...","[sometimes, playing, game, answer, question, c..."
3,I have had a lot of fun with this thing. My 4 ...,1,I have had a lot of fun with this thing My yr ...,have had lot fun with this thing old learns ab...,"[have, had, lot, fun, with, this, thing, old, ...","[lot, fun, thing, old, learns, dinosaurs, cont..."
4,Music,1,Music,Music,[music],[music]


#  Train custom Word2Vec model to learn 50 dimension embedding representation for each word

In [26]:
from gensim.models import Word2Vec

gensim_custom_model = Word2Vec(df['reviews_tokenized_stop'],
                min_count=1,      # Ignores words appearing less than min_count
                vector_size = 50, # Dimensionality of word embeddings
                workers = 2,      # Number of processors (for parallelization)
                window = 5,       # Context window for words during training
                epochs=10)        # Number of epochs training over corpus

print(gensim_custom_model)

Word2Vec<vocab=3766, vector_size=50, alpha=0.025>


In [27]:
words = gensim_custom_model.wv.index_to_key

print(len(words))
print(words)

3766
['love', 'echo', 'great', 'alexa', 'music', 'like', 'use', 'works', 'one', 'easy', 'sound', 'set', 'dot', 'good', 'amazon', 'product', 'get', 'speaker', 'would', 'home', 'play', 'device', 'still', 'time', 'really', 'bought', 'smart', 'new', 'better', 'much', 'prime', 'also', 'quality', 'well', 'day', 'fun', 'work', 'show', 'things', 'far', 'even', 'got', 'devices', 'room', 'thing', 'plus', 'everything', 'house', 'screen', 'ask', 'alarm', 'learning', 'need', 'little', 'buy', 'able', 'weather', 'want', 'could', 'spot', 'lights', 'clock', 'voice', 'using', 'hub', 'nice', 'used', 'know', 'turn', 'video', 'see', 'price', 'many', 'phone', 'light', 'amazing', 'getting', 'purchase', 'think', 'without', 'another', 'listen', 'sounds', 'bedroom', 'app', 'awesome', 'best', 'first', 'setup', 'purchased', 'lot', 'wish', 'control', 'perfect', 'two', 'way', 'connect', 'working', 'playing', 'say', 'questions', 'family', 'features', 'tell', 'speakers', 'make', 'going', 'sure', 'refurbished', 'every

# sample embedding vector

In [28]:
music_embedding = gensim_custom_model.wv['music']

print(len(music_embedding))
print(music_embedding)

50
[ 0.34838137  0.05537323 -0.05509524  0.39532676 -0.10984857 -1.0382049
  0.92975146  1.1771505  -1.4327997   0.15646446 -0.2002896  -1.1365901
  0.2142671   0.6879664  -0.41851646  0.21462607  0.65987724 -0.01674228
 -1.038452   -0.18092236 -0.1249956   0.24861938  1.1374061  -0.4107101
  0.88658947  0.2607965  -0.30064917 -0.69175935 -0.9946816   0.3599123
  0.31808156 -0.13644274 -0.63185054  0.39405724 -0.47715974  0.7507714
  0.44066474  0.20916966 -0.05038728 -0.5492387   0.7483788   0.02995334
 -0.13391656  0.05445005  1.5720347   0.32799768 -0.09397218 -0.8383936
  0.51107264 -0.03857043]


In [30]:
print(len(gensim_custom_model.wv['playing']))
print(gensim_custom_model.wv['playing'])

50
[ 0.26756084  0.01512673 -0.00687519  0.38574207 -0.08325278 -0.9929391
  0.87101984  1.1442493  -1.1987064   0.01201071 -0.17624013 -1.0609804
  0.11627545  0.61908305 -0.4309588   0.31220874  0.51785445 -0.06654853
 -0.92915624 -0.20366707 -0.05652229  0.31415913  1.1275042  -0.35132977
  0.75385857  0.2616182  -0.3500555  -0.6728154  -0.8899745   0.2319342
  0.18866366 -0.06504387 -0.49831912  0.4021138  -0.39645073  0.67396045
  0.47444862  0.12777463 -0.09223744 -0.51486456  0.7636175  -0.00894977
 -0.16428052  0.03787278  1.4175774   0.35875627 -0.13875127 -0.77496815
  0.5238673   0.03848248]


 # Display the words that are most relevant

In [32]:
gensim_custom_model.wv.most_similar('playing')

[('listening', 0.9989573955535889),
 ('see', 0.998916745185852),
 ('sometimes', 0.9987895488739014),
 ('etc', 0.9987472891807556),
 ('heard', 0.9987202882766724),
 ('youtube', 0.9985900521278381),
 ('setting', 0.9985543489456177),
 ('timers', 0.9985453486442566),
 ('shows', 0.9984867572784424),
 ('something', 0.9984714984893799)]

# Aggregted sentence vector for each sentence based on the word vectors

In [41]:
words = set(gensim_custom_model.wv.index_to_key)

In [39]:
# Assuming df_amazon_mod already has a 'verified_reviews' column containing the text
df_amazon_mod['reviews_non_numeric'] = df_amazon_mod['verified_reviews'].apply(clean) # Apply the 'clean' function to remove punctuation and numerics
df_amazon_mod['reviews_len_trim'] = df_amazon_mod.reviews_non_numeric.map(lambda x: ' '.join([w for w in x.split() if len(w)>2])) # Trim out words with two or less characters
df_amazon_mod['reviews_tokenized'] = df_amazon_mod.reviews_len_trim.map(lambda x: x.lower().split()) # Force lowercase and tokenize
df_amazon_mod['reviews_tokenized_stop'] = df_amazon_mod.reviews_tokenized.map(lambda x: [w for w in x if not w in stop_words]) # Remove stopwords


df_amazon_mod['reviews_vect'] = np.array([np.array([gensim_custom_model.wv[i] for i in ls if i in words])
                       for ls in df_amazon_mod['reviews_tokenized_stop']], dtype=object) # Calculate sentence vectors

In [42]:
df_amazon_mod.head()

,verified_reviews,feedback,reviews_non_numeric,reviews_len_trim,reviews_tokenized,reviews_tokenized_stop,reviews_vect
0,Love my Echo!,1,Love my Echo,Love Echo,"[love, echo]","[love, echo]","[[0.24999279, -0.009291827, 0.0076787565, 0.47..."
1,Loved it!,1,Loved it,Loved,[loved],[loved],"[[0.17990361, 0.015443894, 0.007193965, 0.3069..."
2,"Sometimes while playing a game, you can answer...",1,Sometimes while playing a game you can answer ...,Sometimes while playing game you can answer qu...,"[sometimes, while, playing, game, you, can, an...","[sometimes, playing, game, answer, question, c...","[[0.22466925, -0.0061673163, -0.0035426733, 0...."
3,I have had a lot of fun with this thing. My 4 ...,1,I have had a lot of fun with this thing My yr ...,have had lot fun with this thing old learns ab...,"[have, had, lot, fun, with, this, thing, old, ...","[lot, fun, thing, old, learns, dinosaurs, cont...","[[0.26770753, 0.046492707, -0.023606237, 0.343..."
4,Music,1,Music,Music,[music],[music],"[[0.34838137, 0.05537323, -0.055095244, 0.3953..."


In [43]:
vector_size_n_w2v = 50
text_vect_avg = []
for v in df_amazon_mod['reviews_vect']:
    if v.size:
        text_vect_avg.append(v.mean(axis=0))
    else:
        text_vect_avg.append(np.zeros(vector_size_n_w2v, dtype=float)) # the same vector size must be used here as for model training


df_amazon_mod['reviews_vect_avg'] = text_vect_avg
df_amazon_mod.head()

,verified_reviews,feedback,reviews_non_numeric,reviews_len_trim,reviews_tokenized,reviews_tokenized_stop,reviews_vect,reviews_vect_avg
0,Love my Echo!,1,Love my Echo,Love Echo,"[love, echo]","[love, echo]","[[0.24999279, -0.009291827, 0.0076787565, 0.47...","[0.21926592, -0.008649662, -0.04220175, 0.4919..."
1,Loved it!,1,Loved it,Loved,[loved],[loved],"[[0.17990361, 0.015443894, 0.007193965, 0.3069...","[0.17990361, 0.015443894, 0.007193965, 0.30698..."
2,"Sometimes while playing a game, you can answer...",1,Sometimes while playing a game you can answer ...,Sometimes while playing game you can answer qu...,"[sometimes, while, playing, game, you, can, an...","[sometimes, playing, game, answer, question, c...","[[0.22466925, -0.0061673163, -0.0035426733, 0....","[0.21471971, 0.02562269, -0.020447155, 0.36507..."
3,I have had a lot of fun with this thing. My 4 ...,1,I have had a lot of fun with this thing My yr ...,have had lot fun with this thing old learns ab...,"[have, had, lot, fun, with, this, thing, old, ...","[lot, fun, thing, old, learns, dinosaurs, cont...","[[0.26770753, 0.046492707, -0.023606237, 0.343...","[0.21812737, 0.02676491, 0.00074823946, 0.3503..."
4,Music,1,Music,Music,[music],[music],"[[0.34838137, 0.05537323, -0.055095244, 0.3953...","[0.34838137, 0.05537323, -0.055095244, 0.39532..."


# Create clean matrix of just the review embeddings

In [48]:
df_review_embeddings = pd.DataFrame(df_amazon_mod.reviews_vect_avg.tolist())
df_review_embeddings

,0,1,2,3,4,5,6,7,8,9,...,40,41,42,43,44,45,46,47,48,49
0,0.219266,-0.008650,-0.042202,0.491917,0.119259,-0.946958,0.920444,1.104366,-1.086875,-0.004430,...,0.898904,-0.029625,-0.223762,0.117564,1.423212,0.333911,-0.361052,-0.812250,0.574834,0.183869
1,0.179904,0.015444,0.007194,0.306986,0.017888,-0.670883,0.623510,0.761945,-0.815482,0.003359,...,0.594787,-0.006839,-0.140352,0.023142,0.944781,0.201793,-0.157598,-0.528563,0.411910,0.036129
2,0.214720,0.025623,-0.020447,0.365074,-0.059071,-0.817878,0.739698,0.892807,-0.956205,0.044371,...,0.659961,-0.008689,-0.128452,0.020511,1.147628,0.255065,-0.114395,-0.619944,0.449569,0.038493
3,0.218127,0.026765,0.000748,0.350338,-0.047254,-0.851094,0.757516,0.946721,-1.032750,0.004660,...,0.678040,-0.006298,-0.139135,0.028589,1.196697,0.271430,-0.111920,-0.654693,0.466619,0.022370
4,0.348381,0.055373,-0.055095,0.395327,-0.109849,-1.038205,0.929751,1.177150,-1.432800,0.156464,...,0.748379,0.029953,-0.133917,0.054450,1.572035,0.327998,-0.093972,-0.838394,0.511073,-0.038570
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3145,0.124427,0.007699,0.007765,0.245997,0.002624,-0.545975,0.476677,0.619785,-0.653524,0.008128,...,0.466427,-0.013748,-0.111106,0.036053,0.772878,0.187816,-0.100142,-0.434325,0.320170,0.027123
3146,0.204370,0.032325,0.020595,0.322573,-0.086085,-0.724727,0.681540,0.791413,-0.904508,0.007798,...,0.596155,0.016544,-0.123013,-0.002030,1.080022,0.210276,-0.105190,-0.582984,0.403009,0.007789
3147,0.171756,0.031276,-0.008671,0.327964,-0.042083,-0.708689,0.623486,0.766642,-0.827516,0.011052,...,0.575604,-0.011478,-0.120008,0.024375,0.991264,0.219260,-0.114549,-0.541443,0.406460,0.048195
3148,0.164821,0.000947,0.005983,0.304269,0.021390,-0.740450,0.665312,0.846541,-0.878036,-0.024278,...,0.618465,-0.033673,-0.148264,0.044577,1.053800,0.255884,-0.156334,-0.598456,0.437244,0.038998


#  train machine learning models to classify the sentiment of Amazon Alexa reviews

In [50]:
df_Machine_Learning.columns = ['Col_' + str(i+1) for i in range(0, df_Machine_Learning.shape[1])]
df_Machine_Learning

,Col_1,Col_2,Col_3,Col_4,Col_5,Col_6,Col_7,Col_8,Col_9,Col_10,...,Col_41,Col_42,Col_43,Col_44,Col_45,Col_46,Col_47,Col_48,Col_49,Col_50
0,0.219266,-0.008650,-0.042202,0.491917,0.119259,-0.946958,0.920444,1.104366,-1.086875,-0.004430,...,0.898904,-0.029625,-0.223762,0.117564,1.423212,0.333911,-0.361052,-0.812250,0.574834,0.183869
1,0.179904,0.015444,0.007194,0.306986,0.017888,-0.670883,0.623510,0.761945,-0.815482,0.003359,...,0.594787,-0.006839,-0.140352,0.023142,0.944781,0.201793,-0.157598,-0.528563,0.411910,0.036129
2,0.214720,0.025623,-0.020447,0.365074,-0.059071,-0.817878,0.739698,0.892807,-0.956205,0.044371,...,0.659961,-0.008689,-0.128452,0.020511,1.147628,0.255065,-0.114395,-0.619944,0.449569,0.038493
3,0.218127,0.026765,0.000748,0.350338,-0.047254,-0.851094,0.757516,0.946721,-1.032750,0.004660,...,0.678040,-0.006298,-0.139135,0.028589,1.196697,0.271430,-0.111920,-0.654693,0.466619,0.022370
4,0.348381,0.055373,-0.055095,0.395327,-0.109849,-1.038205,0.929751,1.177150,-1.432800,0.156464,...,0.748379,0.029953,-0.133917,0.054450,1.572035,0.327998,-0.093972,-0.838394,0.511073,-0.038570
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3145,0.124427,0.007699,0.007765,0.245997,0.002624,-0.545975,0.476677,0.619785,-0.653524,0.008128,...,0.466427,-0.013748,-0.111106,0.036053,0.772878,0.187816,-0.100142,-0.434325,0.320170,0.027123
3146,0.204370,0.032325,0.020595,0.322573,-0.086085,-0.724727,0.681540,0.791413,-0.904508,0.007798,...,0.596155,0.016544,-0.123013,-0.002030,1.080022,0.210276,-0.105190,-0.582984,0.403009,0.007789
3147,0.171756,0.031276,-0.008671,0.327964,-0.042083,-0.708689,0.623486,0.766642,-0.827516,0.011052,...,0.575604,-0.011478,-0.120008,0.024375,0.991264,0.219260,-0.114549,-0.541443,0.406460,0.048195
3148,0.164821,0.000947,0.005983,0.304269,0.021390,-0.740450,0.665312,0.846541,-0.878036,-0.024278,...,0.618465,-0.033673,-0.148264,0.044577,1.053800,0.255884,-0.156334,-0.598456,0.437244,0.038998


In [51]:
df_Machine_Learning = pd.DataFrame(text_vect_avg)
df_Machine_Learning

,0,1,2,3,4,5,6,7,8,9,...,40,41,42,43,44,45,46,47,48,49
0,0.219266,-0.008650,-0.042202,0.491917,0.119259,-0.946958,0.920444,1.104366,-1.086875,-0.004430,...,0.898904,-0.029625,-0.223762,0.117564,1.423212,0.333911,-0.361052,-0.812250,0.574834,0.183869
1,0.179904,0.015444,0.007194,0.306986,0.017888,-0.670883,0.623510,0.761945,-0.815482,0.003359,...,0.594787,-0.006839,-0.140352,0.023142,0.944781,0.201793,-0.157598,-0.528563,0.411910,0.036129
2,0.214720,0.025623,-0.020447,0.365074,-0.059071,-0.817878,0.739698,0.892807,-0.956205,0.044371,...,0.659961,-0.008689,-0.128452,0.020511,1.147628,0.255065,-0.114395,-0.619944,0.449569,0.038493
3,0.218127,0.026765,0.000748,0.350338,-0.047254,-0.851094,0.757516,0.946721,-1.032750,0.004660,...,0.678040,-0.006298,-0.139135,0.028589,1.196697,0.271430,-0.111920,-0.654693,0.466619,0.022370
4,0.348381,0.055373,-0.055095,0.395327,-0.109849,-1.038205,0.929751,1.177150,-1.432800,0.156464,...,0.748379,0.029953,-0.133917,0.054450,1.572035,0.327998,-0.093972,-0.838394,0.511073,-0.038570
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3145,0.124427,0.007699,0.007765,0.245997,0.002624,-0.545975,0.476677,0.619785,-0.653524,0.008128,...,0.466427,-0.013748,-0.111106,0.036053,0.772878,0.187816,-0.100142,-0.434325,0.320170,0.027123
3146,0.204370,0.032325,0.020595,0.322573,-0.086085,-0.724727,0.681540,0.791413,-0.904508,0.007798,...,0.596155,0.016544,-0.123013,-0.002030,1.080022,0.210276,-0.105190,-0.582984,0.403009,0.007789
3147,0.171756,0.031276,-0.008671,0.327964,-0.042083,-0.708689,0.623486,0.766642,-0.827516,0.011052,...,0.575604,-0.011478,-0.120008,0.024375,0.991264,0.219260,-0.114549,-0.541443,0.406460,0.048195
3148,0.164821,0.000947,0.005983,0.304269,0.021390,-0.740450,0.665312,0.846541,-0.878036,-0.024278,...,0.618465,-0.033673,-0.148264,0.044577,1.053800,0.255884,-0.156334,-0.598456,0.437244,0.038998


In [52]:
final_df = pd.concat([df_amazon_mod[['verified_reviews','feedback']], df_Machine_Learning], axis=1, sort=False)
final_df

,verified_reviews,feedback,0,1,2,3,4,5,6,7,...,40,41,42,43,44,45,46,47,48,49
0,Love my Echo!,1,0.219266,-0.008650,-0.042202,0.491917,0.119259,-0.946958,0.920444,1.104366,...,0.898904,-0.029625,-0.223762,0.117564,1.423212,0.333911,-0.361052,-0.812250,0.574834,0.183869
1,Loved it!,1,0.179904,0.015444,0.007194,0.306986,0.017888,-0.670883,0.623510,0.761945,...,0.594787,-0.006839,-0.140352,0.023142,0.944781,0.201793,-0.157598,-0.528563,0.411910,0.036129
2,"Sometimes while playing a game, you can answer...",1,0.214720,0.025623,-0.020447,0.365074,-0.059071,-0.817878,0.739698,0.892807,...,0.659961,-0.008689,-0.128452,0.020511,1.147628,0.255065,-0.114395,-0.619944,0.449569,0.038493
3,I have had a lot of fun with this thing. My 4 ...,1,0.218127,0.026765,0.000748,0.350338,-0.047254,-0.851094,0.757516,0.946721,...,0.678040,-0.006298,-0.139135,0.028589,1.196697,0.271430,-0.111920,-0.654693,0.466619,0.022370
4,Music,1,0.348381,0.055373,-0.055095,0.395327,-0.109849,-1.038205,0.929751,1.177150,...,0.748379,0.029953,-0.133917,0.054450,1.572035,0.327998,-0.093972,-0.838394,0.511073,-0.038570
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3145,"Perfect for kids, adults and everyone in betwe...",1,0.124427,0.007699,0.007765,0.245997,0.002624,-0.545975,0.476677,0.619785,...,0.466427,-0.013748,-0.111106,0.036053,0.772878,0.187816,-0.100142,-0.434325,0.320170,0.027123
3146,"Listening to music, searching locations, check...",1,0.204370,0.032325,0.020595,0.322573,-0.086085,-0.724727,0.681540,0.791413,...,0.596155,0.016544,-0.123013,-0.002030,1.080022,0.210276,-0.105190,-0.582984,0.403009,0.007789
3147,"I do love these things, i have them running my...",1,0.171756,0.031276,-0.008671,0.327964,-0.042083,-0.708689,0.623486,0.766642,...,0.575604,-0.011478,-0.120008,0.024375,0.991264,0.219260,-0.114549,-0.541443,0.406460,0.048195
3148,Only complaint I have is that the sound qualit...,1,0.164821,0.000947,0.005983,0.304269,0.021390,-0.740450,0.665312,0.846541,...,0.618465,-0.033673,-0.148264,0.044577,1.053800,0.255884,-0.156334,-0.598456,0.437244,0.038998


# Logistic Regression Classifier

In [53]:
from sklearn.linear_model import LogisticRegression
classifier = LogisticRegression()
classifier.fit(df_Machine_Learning, final_df['feedback'])

LogisticRegression()

In [54]:
import pickle as pk
pk.dump(classifier, open('clf_model.pkl', 'wb'))

In [56]:
# Save the model first with the correct file name
gensim_custom_model.save("gensim_embed_model.bin")

# Now load it
w2v_model_reloaded = Word2Vec.load("gensim_embed_model.bin")
vector_size_n_reloaded = 50

print(w2v_model_reloaded)
print(vector_size_n_reloaded)

Word2Vec<vocab=3766, vector_size=50, alpha=0.025>
50


# Predictions on New Test data

In [61]:
new_input = ["Flowers I like to see in the park especially Lily",
             "I like flowers"]

print(new_input[0])
print(new_input[1])

Flowers I like to see in the park especially Lily
I like flowers


In [65]:
new_input_df = pd.DataFrame(new_input, columns=['New_Input'])
new_input_df

,New_Input
0,Flowers I like to see in the park especially Lily
1,I like flowers


In [69]:
new_input_df['reviews_non_numeric'] = new_input_df['New_Input'].map(clean)
new_input_df['reviews_len_trim'] = new_input_df['reviews_non_numeric'].apply(lambda x: ' '.join([w for w in x.split() if len(w)>2]))
new_input_df['reviews_Tokenized'] = new_input_df['reviews_len_trim'].map(lambda x: x.lower().split())
new_input_df['reviews_Tokenized_stop'] = new_input_df['reviews_Tokenized'].map(lambda x: " ".join(x for x in x if x not in stop_words))
new_input_df['reviews_Tokenized_stop'] = new_input_df['reviews_Tokenized_stop'].map(lambda x: x.split())

new_input_df

,New_Input,reviews_non_numeric,reviews_len_trim,reviews_Tokenized,reviews_Tokenized_stop
0,Flowers I like to see in the park especially Lily,Flowers I like to see in the park especially Lily,Flowers like see the park especially Lily,"[flowers, like, see, the, park, especially, lily]","[flowers, like, see, park, especially, lily]"
1,I like flowers,I like flowers,like flowers,"[like, flowers]","[like, flowers]"


In [71]:
new_model = Word2Vec.load("gensim_embed_model.bin") # Assuming 'gensim_embed_model.bin' is the name of your saved model file

new_input_df['new_input_vect'] = new_input_df['reviews_Tokenized_stop'].map(
    lambda ls: np.array([new_model.wv[i] for i in ls if i in words])
)

new_input_df.head()

,New_Input,reviews_non_numeric,reviews_len_trim,reviews_Tokenized,reviews_Tokenized_stop,new_input_vect
0,Flowers I like to see in the park especially Lily,Flowers I like to see in the park especially Lily,Flowers like see the park especially Lily,"[flowers, like, see, the, park, especially, lily]","[flowers, like, see, park, especially, lily]","[[0.3030855, 0.027800487, -0.05017192, 0.46267..."
1,I like flowers,I like flowers,like flowers,"[like, flowers]","[like, flowers]","[[0.3030855, 0.027800487, -0.05017192, 0.46267..."


In [72]:
text_vect_avg = []
for v in new_input_df['new_input_vect']:
    if v.size:
        text_vect_avg.append(v.mean(axis=0))
    else:
        text_vect_avg.append(np.zeros(embed_dimensions, dtype=float))


new_input_df['new_input_vect_avg'] = text_vect_avg
new_input_df

,New_Input,reviews_non_numeric,reviews_len_trim,reviews_Tokenized,reviews_Tokenized_stop,new_input_vect,new_input_vect_avg
0,Flowers I like to see in the park especially Lily,Flowers I like to see in the park especially Lily,Flowers like see the park especially Lily,"[flowers, like, see, the, park, especially, lily]","[flowers, like, see, park, especially, lily]","[[0.3030855, 0.027800487, -0.05017192, 0.46267...","[0.20280404, 0.008088717, -0.026775058, 0.3296..."
1,I like flowers,I like flowers,like flowers,"[like, flowers]","[like, flowers]","[[0.3030855, 0.027800487, -0.05017192, 0.46267...","[0.3030855, 0.027800487, -0.05017192, 0.462671..."


In [73]:
new_input_ml_df = pd.DataFrame(text_vect_avg)
new_input_ml_df

,0,1,2,3,4,5,6,7,8,9,...,40,41,42,43,44,45,46,47,48,49
0,0.202804,0.008089,-0.026775,0.329615,-0.042819,-0.766428,0.698282,0.838139,-0.929590,0.003327,...,0.617426,-0.011544,-0.127841,0.014757,1.100804,0.262382,-0.116849,-0.611411,0.448440,0.022035
1,0.303086,0.027800,-0.050172,0.462672,-0.073019,-1.143435,1.025852,1.205412,-1.369425,-0.017778,...,0.903502,0.026170,-0.159558,-0.000786,1.637995,0.385883,-0.152719,-0.865635,0.660141,0.034359


In [74]:
final_new_input_df = pd.concat([new_input_df['New_Input'], new_input_ml_df], axis=1)
final_new_input_df

,New_Input,0,1,2,3,4,5,6,7,8,...,40,41,42,43,44,45,46,47,48,49
0,Flowers I like to see in the park especially Lily,0.202804,0.008089,-0.026775,0.329615,-0.042819,-0.766428,0.698282,0.838139,-0.929590,...,0.617426,-0.011544,-0.127841,0.014757,1.100804,0.262382,-0.116849,-0.611411,0.448440,0.022035
1,I like flowers,0.303086,0.027800,-0.050172,0.462672,-0.073019,-1.143435,1.025852,1.205412,-1.369425,...,0.903502,0.026170,-0.159558,-0.000786,1.637995,0.385883,-0.152719,-0.865635,0.660141,0.034359


In [75]:
y_pred = classifier.predict(new_input_ml_df)
y_pred

array([1, 1])

In [76]:
final_new_input_df['Prediction'] = y_pred
final_new_input_df

,New_Input,0,1,2,3,4,5,6,7,8,...,41,42,43,44,45,46,47,48,49,Prediction
0,Flowers I like to see in the park especially Lily,0.202804,0.008089,-0.026775,0.329615,-0.042819,-0.766428,0.698282,0.838139,-0.929590,...,-0.011544,-0.127841,0.014757,1.100804,0.262382,-0.116849,-0.611411,0.448440,0.022035,1
1,I like flowers,0.303086,0.027800,-0.050172,0.462672,-0.073019,-1.143435,1.025852,1.205412,-1.369425,...,0.026170,-0.159558,-0.000786,1.637995,0.385883,-0.152719,-0.865635,0.660141,0.034359,1


In [77]:
final_new_input_df[['New_Input', 'Prediction']]

,New_Input,Prediction
0,Flowers I like to see in the park especially Lily,1
1,I like flowers,1
